# Heat map territoriali con Python (esempio concreto)

In questo notebook costruiamo **heat map (mappe di calore)** a partire da dati su un territorio.

L’obiettivo è vedere due situazioni molto comuni:
1. **Heat map su griglia geografica (latitudine/longitudine)**: tipico di meteo, qualità dell’aria, sensori ambientali.
2. **Heat map “tabellare”** (matrice righe/colonne): tipico di dati territoriali aggregati (es. province × giorni).

> Per mantenere il notebook autosufficiente, useremo un **dataset sintetico ma realistico**: simuleremo temperature su una porzione dell’Italia settentrionale (area tra Lombardia/Emilia-Romagna/Veneto) su una griglia regolare.


## Librerie

- **NumPy**: per creare griglie e calcolare valori.
- **Matplotlib**: per disegnare le heat map.
- **Pandas**: per costruire una tabella e fare pivot (nella seconda parte).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Dati di esempio: una griglia su un’area geografica

Una heat map “territoriale” spesso nasce da punti su una superficie.

Qui creiamo una griglia di:
- **latitudine** (asse verticale)
- **longitudine** (asse orizzontale)

Poi simuleremo una variabile (temperatura) che cambia:
- con la latitudine (più a nord = più fresco, in media)
- con una “macchia” calda (es. zona urbana/valle calda)
- con un po’ di rumore (per rendere il tutto meno perfetto)

Questa logica è molto simile a ciò che succede con dati reali (sensori, interpolazioni, modelli meteo).


In [ ]:
# Intervallo geografico (approssimativo) per un'area del Nord Italia
# Longitudine: 9.0 - 12.5  (circa tra Lombardia e Veneto)
# Latitudine: 44.0 - 46.5  (circa tra Emilia-Romagna e Trentino)
lon = np.linspace(9.0, 12.5, 120)
lat = np.linspace(44.0, 46.5, 100)

LON, LAT = np.meshgrid(lon, lat)

# Simulazione temperatura (°C) con un gradiente nord-sud + una "bolla" calda + rumore
np.random.seed(7)

gradiente = 30 - 3.5*(LAT - 44.0)  # più a nord (LAT maggiore) -> temperatura più bassa
bolla_calda = 4.0 * np.exp(-(((LON - 10.3)/0.45)**2 + ((LAT - 45.1)/0.35)**2))  # hotspot
rumore = np.random.normal(0, 0.35, size=LON.shape)

T = gradiente + bolla_calda + rumore

T.min(), T.max()

## 1) Heat map su griglia geografica (lat/lon)

### Idea
Una heat map su griglia è, in pratica, un’immagine in cui ogni cella ha un valore.

Con Matplotlib, due approcci tipici sono:
- `imshow`: veloce, ma meno “geografico” se non stai attento alle coordinate
- `pcolormesh`: ottimo quando hai coordinate reali (lat/lon), perché rispetta la griglia

Qui usiamo `pcolormesh`, così gli assi mostrano direttamente **longitudine e latitudine**.


In [ ]:
plt.figure(figsize=(10, 6))

# pcolormesh usa una griglia: X=longitudine, Y=latitudine, Z=valore
mappa = plt.pcolormesh(LON, LAT, T, shading="auto")

plt.xlabel("Longitudine (°E)")
plt.ylabel("Latitudine (°N)")
plt.title("Heat map territoriale: temperatura simulata (Nord Italia)")

# Barra colori: serve per leggere i valori
plt.colorbar(mappa, label="Temperatura (°C)")

plt.grid(True, alpha=0.25)
plt.show()

## Aggiungere “punti di riferimento” (città) sulla heat map

Quando una heat map è geografica, spesso è utile mettere qualche riferimento visivo:
ad esempio le coordinate (approssimate) di alcune città.

Qui aggiungiamo:
- Milano (9.19, 45.46)
- Bologna (11.34, 44.49)
- Verona (10.99, 45.44)

> Le coordinate qui sono usate solo come riferimento grafico, non come dataset ufficiale.


In [ ]:
citta = {
    "Milano": (9.19, 45.46),
    "Bologna": (11.34, 44.49),
    "Verona": (10.99, 45.44),
}

plt.figure(figsize=(10, 6))
mappa = plt.pcolormesh(LON, LAT, T, shading="auto")
plt.colorbar(mappa, label="Temperatura (°C)")

for nome, (cx, cy) in citta.items():
    plt.scatter([cx], [cy], s=55)
    plt.annotate(nome, (cx, cy), textcoords="offset points", xytext=(6, 6))

plt.xlabel("Longitudine (°E)")
plt.ylabel("Latitudine (°N)")
plt.title("Heat map con punti di riferimento (città)")
plt.grid(True, alpha=0.25)
plt.show()

## 2) Heat map “tabellare”: province × giorni (esempio territoriale)

### Perché serve anche questa?
Non sempre i dati territoriali sono su griglia lat/lon.
Spesso sono aggregati per unità amministrative (province, comuni, regioni) e per tempo (giorni, mesi).

Una rappresentazione tipica è una **matrice**:
- righe = province
- colonne = giorni
- valore = temperatura media, pioggia, inquinamento, ecc.

In pratica: trasformiamo un elenco di misure in una tabella pivot e poi la disegniamo.


In [ ]:
# Province di esempio (scelte per coerenza con l'area "Nord Italia" del nostro esempio)
province = ["Milano", "Bologna", "Verona", "Mantova", "Modena", "Padova"]

# 14 giorni (due settimane)
giorni = pd.date_range("2026-01-01", periods=14, freq="D")

# Simuliamo una temperatura media provinciale: base diversa per provincia + oscillazione giornaliera
rng = np.random.default_rng(12)

base_prov = {
    "Milano": 6.5,
    "Bologna": 7.5,
    "Verona": 6.8,
    "Mantova": 6.9,
    "Modena": 7.3,
    "Padova": 6.6,
}

records = []
for prov in province:
    for i, g in enumerate(giorni):
        oscillazione = 1.8 * np.sin(2*np.pi*i/7)  # ciclo settimanale
        rum = rng.normal(0, 0.35)
        temp = base_prov[prov] + oscillazione + rum
        records.append({"Provincia": prov, "Giorno": g, "TempMedia": temp})

df = pd.DataFrame(records)
df.head()

### Pivot: trasformare la lista di misure in matrice

Una pivot ci permette di ottenere una tabella del tipo:

- indice (righe): Provincia  
- colonne: Giorno  
- valori: TempMedia  

Così la heat map diventa immediata.


In [ ]:
pivot = df.pivot(index="Provincia", columns="Giorno", values="TempMedia")
pivot

### Disegnare la heat map tabellare

Qui usiamo `imshow` perché la tabella è già una matrice.
Aggiungiamo:
- etichette delle province
- etichette dei giorni (formato breve)
- colorbar per leggere i valori


In [ ]:
mat = pivot.to_numpy()

plt.figure(figsize=(12, 4))
img = plt.imshow(mat, aspect="auto")

plt.title("Heat map tabellare: temperatura media (province × giorni)")
plt.xlabel("Giorni")
plt.ylabel("Province")

# tick asse Y: province
plt.yticks(np.arange(len(pivot.index)), pivot.index)

# tick asse X: giorni in formato gg/mm
etichette_giorni = [d.strftime("%d/%m") for d in pivot.columns]
plt.xticks(np.arange(len(pivot.columns)), etichette_giorni, rotation=45, ha="right")

plt.colorbar(img, label="Temperatura media (°C)")
plt.grid(False)
plt.tight_layout()
plt.show()